## Accelerate Inference: Neural Network Pruning

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.10.0


In [57]:
# ! pip install --user install tensorflow==2.10.0 
# print(tf.version.VERSION)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.1 MB 5.8 kB/s 
     |████████████████████████████████| 1.7 MB 58.0 MB/s 
     |████████████████████████████████| 438 kB 73.6 MB/s 
     |████████████████████████████████| 5.9 MB 57.9 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


2.9.2


In [2]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [3]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [5]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/root/.local/lib/python3.8/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


703/703 [==============================] - 9s 8ms/step - loss: 1.5264 - accuracy: 0.2988 - val_loss: 1.4012 - val_accuracy: 0.3921
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3554 - accuracy: 0.4154 - val_loss: 1.2808 - val_accuracy: 0.4657
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2802 - accuracy: 0.4638 - val_loss: 1.2251 - val_accuracy: 0.5006
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2405 - accuracy: 0.4815 - val_loss: 1.1903 - val_accuracy: 0.5152
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2025 - accuracy: 0.5072 - val_loss: 1.1888 - val_accuracy: 0.5109
Epoch 6/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1726 - accuracy: 0.5197 - val_loss: 1.1210 - val_accuracy: 0.5347
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1384 - accuracy: 0.5402 - val_loss: 1.1011 - val_accuracy: 0.5552
Epoch 8/50
703/70

In [6]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 9ms/step - loss: 0.7240 - accuracy: 0.7196


In [19]:
# perform pruning here

import copy

def prune(model, prune_rate):

  # Collect all convolutional layers
  conv_layers = []
  num_layers = len(model.layers)
  for layer in range(num_layers):
    name = model.layers[layer].name
    if name.startswith("conv"):
      conv_layers.append(layer) 

  # Iterate over weights again and set to zero when below threshold 
  for c_l in conv_layers:

    # Collect weights in layer
    conv_layer = model.layers[c_l]
    w_l = conv_layer.get_weights()[0] # shape h,w,c,f
    # w_l_all = np.sort(w_l.flatten())

    # Threshold = (prune_rate)th percentile of the ordered weights
    threshold = np.percentile(w_l, prune_rate*100)  

    # Prune by setting to zero
    w_l[w_l < threshold] = 0

    # Set new weights
    w_copy = copy.deepcopy(conv_layer.get_weights())
    w_copy[0] = w_l
    model.layers[c_l].set_weights(w_copy)

# get the weights 
# weights = model.get_weights()
# print(weights[0].shape)

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# weights[7][:10]=0
# model.set_weights(weights)


In [20]:
#copy model + prune
model2 = keras.models.clone_model(model)
model2.build() # replace 10 with number of variables in input layer
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.set_weights(model.get_weights())

prune(model2, 0.5)

c = len(model2.layers)
total_weights = 0
zero_weights = 0
for i in range(c):
  name = model2.layers[i].name
  if(name.startswith("conv")):
    w1 = model2.layers[i].get_weights()[0]
    for weight in w1:
      total_weights += weight.size
      zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)

65376 33867


In [21]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)

/root/.local/lib/python3.8/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


20/20 [==============================] - 0s 5ms/step - loss: 1.5798 - accuracy: 0.2570


In [22]:
#train again with pruned

model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model2.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 5s 6ms/step - loss: 1.1636 - accuracy: 0.5279 - val_loss: 0.9849 - val_accuracy: 0.6048
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.0052 - accuracy: 0.6056 - val_loss: 0.9376 - val_accuracy: 0.6281
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 0.9481 - accuracy: 0.6293 - val_loss: 0.9028 - val_accuracy: 0.6444
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 0.9125 - accuracy: 0.6419 - val_loss: 0.8491 - val_accuracy: 0.6725
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 0.8786 - accuracy: 0.6623 - val_loss: 0.8442 - val_accuracy: 0.6733
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 0.8537 - accuracy: 0.6723 - val_loss: 0.8336 - val_accuracy: 0.6717
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 0.8322 - accuracy: 0.6816 - val_loss: 0.8355 - val_accuracy: 0.6741
Epoch 

In [28]:
# evaluate again to see how the accuracy changes
results = model2.evaluate(val_images, val_labels, batch_size=128)
print(results)

20/20 [==============================] - 0s 5ms/step - loss: 0.7125 - accuracy: 0.7406
[0.7124612927436829, 0.7405940890312195]


In [25]:
#get num_zero + score
total_weights = 0
zero_weights = 0
all_model2_weights = model2.get_weights()
for weight in all_model2_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)
print(( 0.7406 + zero_weights/total_weights)/2)

592933 14841
0.38281490471942026


In [43]:
#get num_zero + score
total_weights = 0
zero_weights = 0
all_model_weights = model.get_weights()
for weight in all_model_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(total_weights, zero_weights)
print(( 0.7271 + zero_weights/total_weights)/2)

592933 0
0.36355


In [31]:
sparsity = [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95]

# sparsity = [0, 0.10]

mag_spar = []
mag_acc = []

for s in sparsity:
  model_s = keras.models.clone_model(model)
  prune(model_s, s)

  model_s.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model_s.fit(train_images, train_labels, batch_size=32, epochs=50, 
                      validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

  results = model_s.evaluate(val_images, val_labels, batch_size=128)
  mag_acc.append(results[1])

  total_weights = 0
  zero_weights = 0
  all_model_weights = model_s.get_weights()
  for weight in all_model_weights:
    total_weights += weight.size
    zero_weights += np.sum(weight == 0)

  mag_spar.append(zero_weights/total_weights)

Epoch 1/50
703/703 [==============================] - 5s 6ms/step - loss: 1.5162 - accuracy: 0.3097 - val_loss: 1.3872 - val_accuracy: 0.3964
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3537 - accuracy: 0.4186 - val_loss: 1.2729 - val_accuracy: 0.4570
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2823 - accuracy: 0.4595 - val_loss: 1.2237 - val_accuracy: 0.4919
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2436 - accuracy: 0.4818 - val_loss: 1.1855 - val_accuracy: 0.5152
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2120 - accuracy: 0.4981 - val_loss: 1.1605 - val_accuracy: 0.5168
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1731 - accuracy: 0.5218 - val_loss: 1.1808 - val_accuracy: 0.5125
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1487 - accuracy: 0.5358 - val_loss: 1.1356 - val_accuracy: 0.5307
Epoch 

In [32]:
print(mag_spar)
print(mag_acc)

[0.0019951664016001807, 0.012362273646432227, 0.022729380891264275, 0.03455702414944015, 0.044924131394272204, 0.05728640504070443, 0.06765351228553648, 0.07802061953036853, 0.08984826278854441, 0.10021537003337645, 0.10632229948409011]
[0.7017821669578552, 0.7152475118637085, 0.6914851665496826, 0.6716831922531128, 0.6685148477554321, 0.6273267269134521, 0.5948514938354492, 0.569504976272583, 0.5172277092933655, 0.437227725982666, 0.19207920134067535]


In [36]:
model_final = keras.models.clone_model(model)
prune(model_final, 0.3)

model_final.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
history = model_final.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

results = model_final.evaluate(val_images, val_labels, batch_size=128)

Epoch 1/50


/root/.local/lib/python3.8/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


703/703 [==============================] - 5s 6ms/step - loss: 1.5726 - accuracy: 0.2696 - val_loss: 1.4807 - val_accuracy: 0.3620
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.4199 - accuracy: 0.3835 - val_loss: 1.3320 - val_accuracy: 0.4289
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3447 - accuracy: 0.4279 - val_loss: 1.2772 - val_accuracy: 0.4550
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3010 - accuracy: 0.4473 - val_loss: 1.2386 - val_accuracy: 0.4745
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2685 - accuracy: 0.4639 - val_loss: 1.2092 - val_accuracy: 0.4966
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2430 - accuracy: 0.4787 - val_loss: 1.2180 - val_accuracy: 0.4863
Epoch 7/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2123 - accuracy: 0.4969 - val_loss: 1.1585 - val_accuracy: 0.5251
Epoch 8/50
703/70

In [38]:
total_weights = 0
zero_weights = 0
all_model_weights = model_final.get_weights()
for weight in all_model_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

print(results[1], zero_weights / total_weights)
print((0.6713 + zero_weights/total_weights)/2)

0.6712871193885803 0.03455702414944015
0.3529285120747201


In [39]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model_final.save_weights("my_model_weights_3.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights_3.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>